# PGEN(Prompt-based domain text Generation)
사전 학습된 언어 모델(GPT-2 등)을 활용하여 도메인 내 텍스트 데이터를 대규모로 생성하는 방법
- [고려대학교 구름](https://github.com/nlpai-lab/KULLM)

## 0. 라이브러리 및 데이터 불러오기

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device('cpu')

NameError: name 'torch' is not defined

In [4]:
import numpy as np
import pandas as pd
#from torch.utils.data import Dataset, DataLoader
#import torch
#import torch.optim as optim

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!pip install utils

  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13905 sha256=f593b44a8cc272f92c0e5e91a99396a6393f3870fc04de973aa83a3930da0e2a
  Stored in directory: /root/.cache/pip/wheels/b8/39/f5/9d0ca31dba85773ececf0a7f5469f18810e1c8a8ed9da28ca7
Successfully built utils


## 1. 데이터 불러오기

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/StoneMaenggu/coda-modeling/main/Gloss2Text/Dataset/NIASL2020_LLM.csv')
df = df[df.g_type=='SEN']
df = df[['start_end_time', 'gloss']]
df['gloss'] = df['gloss'].astype(str).apply(lambda x: x.replace('|', ' '))
df.text = ''
df

,start_end_time,gloss
48000,"[[1.975, 2.856], [2.989, 3.489], [3.608, 4.203...",기차 곳 내리다 맞다
48001,"[[1.626, 2.389], [2.477, 2.977], [3.09, 3.59],...",편지 저기 곳 다음
48002,"[[2.362, 3.974]]",위험
48003,"[[1.856, 2.638], [2.799, 3.388], [3.557, 4.186...",스타벅스 곳 저기 전 도착 차내리다
48004,"[[1.213, 1.899], [2.107, 2.607], [2.843, 3.343...",여기 송파 지하철 맞다
...,...,...
79995,"[[2.028, 2.528], [2.647, 3.483]]",그남자 맞다
79996,"[[1.959, 2.668], [2.782, 3.282], [3.42, 4.676]]",손바닥찍다 가다 내리다
79997,"[[1.847, 2.096], [2.41, 2.774], [3.095, 4.134]]",핸드폰 잃어버리다 찾다
79998,"[[1.407, 2.179], [2.37, 3.278]]",가방 실종


## 2. LLM 불러오기

### git clone

In [7]:
!git clone -b kullm_v2 https://github.com/nlpai-lab/KULLM.git

fatal: destination path 'KULLM' already exists and is not an empty directory.


In [8]:
!cd KULLM

In [9]:
#!conda create -n KULLM python=3.10
!pip install -U torch transformers tokenizers accelerate jupyter

  Using cached transformers-4.44.0-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.20.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.44.0-py3-none-any.whl (9.5 MB)
Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


In [12]:
!pip install torch transformers==4.38.2 accelerate

  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
  Using cached tokenizers-0.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)
Using cached tokenizers-0.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.0
    Uninstalling transformers-4.44.0:
      Successfully uninstalled transformers-4.44.0


In [2]:
!pip install utils

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

import sys
sys.path.append('/content/KULLM/utils')
from prompter import Prompter

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device('cpu')

In [8]:
MODEL = "nlpai-lab/kullm-polyglot-5.8b-v2"

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,).to(device)

model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/36.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/3.73G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30080, 4096)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-27): 28 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)

### model

In [10]:
pipe = pipeline("text-generation", model=model, tokenizer=MODEL, device=0)
prompter = Prompter("/content/KULLM/templates/kullm")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [43]:
def infer(instruction=instruction, input_text=""):
    prompt = prompter.generate_prompt(instruction, input_text)
    output = pipe(prompt, max_length=512, temperature=0.2, num_beams=5, eos_token_id=2)
    s = output[0]["generated_text"]
    result = prompter.get_response(s)
    return result

In [47]:
instruction = '단어 몇개만으로 문장을 만들어줘. 질문에 대답하지 않고 문장을 생성해줘. 명사나 동사의 원형을 입력하면 그에 맞춰 문장을 생성하면 돼. 동사의 의미는 그대로 반영하되 문법적 요소를 고려해 알맞은 문장을 생성해줘. 예시는 다음과 같아. 1. [남자 생각 궁금 물어보다] : [그의 의사는 어떤지 물어보았다] 2. [너 마음 궁금]:[네 의사가 궁금해]'
              # +3. [아이 잃어버리다 실종]:[어린애를 잃어버렸습니다.]
              # '4. [목 마르다 물 원하다]:[목 말라서 물을 마시고 싶어.]
              # [친구 싸우다 싫다]:[그 친구랑 싸워서 싫어]'

In [48]:
infer(input_text="누구 너")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 14.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 9.06 MiB is free. Process 122625 has 14.74 GiB memory in use. Of the allocated memory 14.09 GiB is allocated by PyTorch, and 532.21 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)